# Convolutional Neural Networks For Image Recognition

## Setup and Imports

In [1]:
# Setup
from __future__ import division, print_function, unicode_literals

# Imports
import numpy as np
import numpy.random as rnd
import os

# To make make consistent across code blocks
rnd.seed(42)

# Ploting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Saving Parameters
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "data", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

**Import Keras**

In [2]:
import keras as kr

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


# Convolutional Neural Network

## Import Dataset

In [3]:
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

In [4]:
def to_one_hot(X):
    n_values = np.max(X) + 1
    y = np.eye(n_values)[X[:,0]]
    return y

y_train_adj = to_one_hot(y_train)
y_test_adj  = to_one_hot(y_test)

In [5]:
print(np.shape(x_train))

(50000, 32, 32, 3)


## Coarse Training

** Defining Structure**

In [6]:
def indep_layers(X):
    net = Conv2D(6, 2, strides=1, activation='elu', padding='same')(X)
    net = Dropout(0.25)(net)
    net = Conv2D(6, 2, strides=1, activation='elu', padding='same')(net)
    net = Dropout(0.25)(net)
    net = Conv2D(6, 2, strides=1, activation='elu', padding='same')(net)
    net = Flatten()(net)

    net = Dense(256, activation='elu')(net)
    net = Dropout(0.50)(net)
    net = Dense(128, activation='elu')(net)
    net = Dropout(0.25)(net)
    net = Dense(100, activation='softmax')(net)
    
    return net

In [7]:
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Dropout, Multiply, Add, Dot, RepeatVector, Reshape
from keras.optimizers import SGD
from keras.models import Model

coarse_categories = 20
fine_categories = 100
sigma = .01

input_img = Input(shape=(32, 32, 3), dtype='float32', name='main_input')

## SHARED LAYERS
net_shared = Conv2D(24, (4, 4), strides=(1, 1), padding='same', activation='elu')(input_img)
net_shared = MaxPooling2D((2, 2), padding='same')(net_shared)
net_shared = Dropout(0.25)(net_shared)
net_shared = Conv2D(12, (2, 2), strides=(1, 1), padding='same', activation='elu')(net_shared)
net_shared = MaxPooling2D(pool_size=2, strides=2, padding='same')(net_shared)
shared_output = Dropout(0.25)(net_shared)

# COARSE CLASSIFIER
# Get the coarse predictions from fine
output_c = indep_layers(shared_output)

base_model = Model(inputs=input_img, outputs=output_c)

base_model.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
base_model.fit(x_train, y_train_adj, epochs=300, batch_size=256, validation_split=.1)
base_model.save_weights('data/models/base_model_'+str(index))

Train on 45000 samples, validate on 5000 samples
Epoch 1/300
45000/45000 [==============================] - 52s - loss: 5.4598 - acc: 0.0101 - val_loss: 4.5959 - val_acc: 0.0138
Epoch 2/300
45000/45000 [==============================] - 52s - loss: 4.6275 - acc: 0.0195 - val_loss: 4.5980 - val_acc: 0.0244
Epoch 3/300
45000/45000 [==============================] - 51s - loss: 4.3041 - acc: 0.0438 - val_loss: 4.3275 - val_acc: 0.0428
Epoch 4/300
45000/45000 [==============================] - 52s - loss: 4.0954 - acc: 0.0700 - val_loss: 4.1629 - val_acc: 0.0652
Epoch 5/300
45000/45000 [==============================] - 52s - loss: 3.9572 - acc: 0.0850 - val_loss: 4.0553 - val_acc: 0.0798
Epoch 6/300
45000/45000 [==============================] - 52s - loss: 3.8501 - acc: 0.1013 - val_loss: 3.8578 - val_acc: 0.1128
Epoch 7/300
45000/45000 [==============================] - 52s - loss: 3.7652 - acc: 0.1133 - val_loss: 3.9050 - val_acc: 0.1070
Epoch 8/300
45000/45000 [=======================

45000/45000 [==============================] - 567s - loss: 3.0668 - acc: 0.2407 - val_loss: 2.8865 - val_acc: 0.2784
Epoch 64/300
45000/45000 [==============================] - 354s - loss: 3.0631 - acc: 0.2405 - val_loss: 2.8972 - val_acc: 0.2838
Epoch 65/300
45000/45000 [==============================] - 328s - loss: 3.0507 - acc: 0.2428 - val_loss: 2.8973 - val_acc: 0.2814
Epoch 66/300
45000/45000 [==============================] - 956s - loss: 3.0530 - acc: 0.2383 - val_loss: 2.9168 - val_acc: 0.2756
Epoch 67/300
45000/45000 [==============================] - 1614s - loss: 3.0541 - acc: 0.2407 - val_loss: 2.9190 - val_acc: 0.2826
Epoch 68/300
45000/45000 [==============================] - 712s - loss: 3.0451 - acc: 0.2443 - val_loss: 2.8155 - val_acc: 0.2984
Epoch 69/300
45000/45000 [==============================] - 725s - loss: 3.0457 - acc: 0.2431 - val_loss: 2.9079 - val_acc: 0.2776
Epoch 70/300
45000/45000 [==============================] - 1127s - loss: 3.0451 - acc: 0.2423 

## Fine-Tuning

**Define New Layers**

In [8]:
from __future__ import absolute_import

from keras.layers import initializers, regularizers, constraints
from keras.engine.topology import Layer, InputSpec
from keras import backend as K
from keras.legacy import interfaces

class Threshold(Layer):
    """Thresholded Rectified Linear Unit.
    It follows:
    `f(x) = 1 for x > theta`,
    `f(x) = 0 otherwise`.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    # Output shape
        Same shape as the input.
    # Arguments
        theta: float >= 0. Threshold value.
    """

    def __init__(self, theta=.5, **kwargs):
        super(Threshold, self).__init__(**kwargs)
        self.theta = K.cast_to_floatx(theta)

    def call(self, inputs, mask=None):
        return K.cast(K.greater(inputs, self.theta), K.floatx())

    def get_config(self):
        config = {'theta': float(self.theta)}
        base_config = super(Threshold, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [9]:
def calc_inputs():
    return [Input(shape=([100]), dtype='float32', name='c0'), \
        Input(shape=([100]), dtype='float32', name='c1'), \
        Input(shape=([100]), dtype='float32', name='c2'), \
        Input(shape=([100]), dtype='float32', name='c3'), \
        Input(shape=([100]), dtype='float32', name='c4'), \
        Input(shape=([100]), dtype='float32', name='c5'), \
        Input(shape=([100]), dtype='float32', name='c6'), \
        Input(shape=([100]), dtype='float32', name='c7'), \
        Input(shape=([100]), dtype='float32', name='c8'), \
        Input(shape=([100]), dtype='float32', name='c9'), \
        Input(shape=([100]), dtype='float32', name='c10'), \
        Input(shape=([100]), dtype='float32', name='c11'), \
        Input(shape=([100]), dtype='float32', name='c12'), \
        Input(shape=([100]), dtype='float32', name='c13'), \
        Input(shape=([100]), dtype='float32', name='c14'), \
        Input(shape=([100]), dtype='float32', name='c15'), \
        Input(shape=([100]), dtype='float32', name='c16'), \
        Input(shape=([100]), dtype='float32', name='c17'), \
        Input(shape=([100]), dtype='float32', name='c18'), \
        Input(shape=([100]), dtype='float32', name='c19')]

In [10]:
def calc_prob_ave(fine_pred, fine_to_coarse):
    coarse_pred = Dot(1)([fine_pred,fine_to_coarse])
    coarse_pred_adj = RepeatVector(100)(coarse_pred)
    coarse_pred_adj = Flatten()(coarse_pred_adj)
    prob_ave = Multiply()([fine_to_coarse, coarse_pred_adj])

    return prob_ave

def calc_threshold(fine_pred, fine_to_coarse, shared_layer):
    coarse_pred = Dot(1)([fine_pred, fine_to_coarse])
    coarse_pred_thresholded = Threshold()(coarse_pred)
    threshold_adj = RepeatVector(12*8*8)(coarse_pred_thresholded)
    threshold_adj = Reshape((8, 8, 12))(threshold_adj)
    shared_layer_adj = Multiply()([shared_layer, threshold_adj])
    
    return shared_layer_adj

In [11]:
c = calc_inputs()

for i in range(20):
    thresholded = calc_threshold(base_model.output, c[i], shared_output)
    fine_output = indep_layers(thresholded)

    prob_ave = calc_prob_ave(output_c, c[i])

    weighted = Multiply()([fine_output, prob_ave])
    
    if i==0: output_f = weighted
    else: output_f = Add()([weighted, output_f])

model = Model(inputs=[base_model.input, c[0], c[1], c[2], c[3], c[4], c[5], c[6], c[7], c[8], c[9], c[10], c[11], c[12], c[13], c[14], c[15], c[16], c[17], c[18], c[19]], 
              outputs=output_f)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer= 'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

**Load Coarse-To-Fine Mappings**

In [ ]:
# Get Coarse to Fine Mappings
C2K = np.loadtxt('C2K.txt', dtype=int)
C2K_dot = np.zeros((20,100))
for i in range(len(C2K)):
    C2K_dot[C2K[i], i] = 1

#np.set_printoptions(threshold=np.inf)
#print(C2K_dot)

**Fit Training Data**

In [ ]:
for i in range(len(x_train)):
    for j in range(20):
        ck[j,i] = C2K_dot[j]

index= 0
step = 25
stop = 500

while index < stop:
    model.fit([x_train, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
              y_train_adj, epochs=step, batch_size=256, validation_split=.1)
    model.save_weights('data/models/layer_'+str(index))
    index += step

**Fit on dataset delayed (for use in browsers)**

This is a complex network and the dataset is not small either so this may occasionally crash if run in a browser. 

In the case that it does, use this function to incrementally train the network 5 (# can be changed if necessary), save the model and then reload it for the next increment of training.

In [68]:
while index < 300:
    model.load_weights('data/models/layer_'+str(index), by_name=False)
    index += step
    model.fit([x_train, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
              y_train_adj, epochs=index+step, batch_size=64, initial_epoch=index, validation_split=.1)
    model.save_weights('data/models/layer_'+str(index))

Train on 45000 samples, validate on 5000 samples
Epoch 26/50
45000/45000 [==============================] - 170s - loss: 4.6786 - acc: 0.0108 - val_loss: 4.6034 - val_acc: 0.0122
Epoch 27/50
45000/45000 [==============================] - 171s - loss: 4.6714 - acc: 0.0107 - val_loss: 4.6022 - val_acc: 0.0090
Epoch 28/50
45000/45000 [==============================] - 170s - loss: 4.6674 - acc: 0.0106 - val_loss: 4.6029 - val_acc: 0.0112
Epoch 29/50
45000/45000 [==============================] - 169s - loss: 4.6663 - acc: 0.0106 - val_loss: 4.6007 - val_acc: 0.0092
Epoch 30/50
45000/45000 [==============================] - 169s - loss: 4.6614 - acc: 0.0109 - val_loss: 4.6021 - val_acc: 0.0086
Epoch 31/50
45000/45000 [==============================] - 175s - loss: 4.6592 - acc: 0.0105 - val_loss: 4.5997 - val_acc: 0.0130
Epoch 32/50
45000/45000 [==============================] - 172s - loss: 4.6592 - acc: 0.0110 - val_loss: 4.5989 - val_acc: 0.0130
Epoch 33/50
45000/45000 [================

45000/45000 [==============================] - 175s - loss: 4.4657 - acc: 0.0214 - val_loss: 4.4111 - val_acc: 0.0220
Epoch 149/150
45000/45000 [==============================] - 175s - loss: 4.4614 - acc: 0.0204 - val_loss: 4.4022 - val_acc: 0.0188
Epoch 150/150
45000/45000 [==============================] - 175s - loss: 4.4643 - acc: 0.0199 - val_loss: 4.4131 - val_acc: 0.0180
Train on 45000 samples, validate on 5000 samples
Epoch 151/175
45000/45000 [==============================] - 174s - loss: 4.4628 - acc: 0.0210 - val_loss: 4.4010 - val_acc: 0.0224
Epoch 152/175
45000/45000 [==============================] - 175s - loss: 4.4589 - acc: 0.0207 - val_loss: 4.4013 - val_acc: 0.0190
Epoch 153/175
45000/45000 [==============================] - 174s - loss: 4.4594 - acc: 0.0214 - val_loss: 4.4043 - val_acc: 0.0186
Epoch 154/175
45000/45000 [==============================] - 176s - loss: 4.4565 - acc: 0.0214 - val_loss: 4.4328 - val_acc: 0.0180
Epoch 155/175
45000/45000 [==============

45000/45000 [==============================] - 176s - loss: 4.3981 - acc: 0.0242 - val_loss: 4.4539 - val_acc: 0.0122
Epoch 210/225
45000/45000 [==============================] - 174s - loss: 4.3937 - acc: 0.0241 - val_loss: 4.3866 - val_acc: 0.0170
Epoch 211/225
45000/45000 [==============================] - 175s - loss: 4.3916 - acc: 0.0240 - val_loss: 4.3505 - val_acc: 0.0232
Epoch 212/225
45000/45000 [==============================] - 175s - loss: 4.3927 - acc: 0.0250 - val_loss: 4.3570 - val_acc: 0.0208
Epoch 213/225
45000/45000 [==============================] - 175s - loss: 4.3907 - acc: 0.0246 - val_loss: 4.3454 - val_acc: 0.0224
Epoch 214/225
45000/45000 [==============================] - 175s - loss: 4.3913 - acc: 0.0227 - val_loss: 4.3433 - val_acc: 0.0226
Epoch 215/225
45000/45000 [==============================] - 175s - loss: 4.3914 - acc: 0.0245 - val_loss: 4.3622 - val_acc: 0.0244
Epoch 216/225
45000/45000 [==============================] - 175s - loss: 4.3888 - acc: 0.

In [76]:
coarse_layers = ['conv2d_327', 'dropout_417', 'conv2d_328', 'dropout_418', 'conv2d_329', 'flatten_154', 'dense_271', 'dropout_419', 'dense_272', 'dropout_420']

model.load_weights('data/models/layer_300', 'conv2d_327', 'conv2d_328')

TypeError: load_weights() takes from 2 to 3 positional arguments but 4 were given

In [69]:
while index < 500:
    model.load_weights('data/models/layer_'+str(index), by_name=False)
    index += step
    model.fit([x_train, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
              y_train_adj, epochs=index+step, batch_size=64, initial_epoch=index, validation_split=.1)
    model.save_weights('data/models/layer_'+str(index))

Train on 45000 samples, validate on 5000 samples
Epoch 326/350
45000/45000 [==============================] - 175s - loss: 4.2875 - acc: 0.0311 - val_loss: 4.3052 - val_acc: 0.0236
Epoch 327/350
45000/45000 [==============================] - 175s - loss: 4.2863 - acc: 0.0305 - val_loss: 4.2905 - val_acc: 0.0272
Epoch 328/350
45000/45000 [==============================] - 175s - loss: 4.2870 - acc: 0.0313 - val_loss: 4.3172 - val_acc: 0.0240
Epoch 329/350
45000/45000 [==============================] - 176s - loss: 4.2857 - acc: 0.0305 - val_loss: 4.3033 - val_acc: 0.0244
Epoch 330/350
45000/45000 [==============================] - 172s - loss: 4.2854 - acc: 0.0319 - val_loss: 4.3024 - val_acc: 0.0260
Epoch 331/350
45000/45000 [==============================] - 175s - loss: 4.2866 - acc: 0.0304 - val_loss: 4.3346 - val_acc: 0.0210
Epoch 332/350
40896/45000 [==========================>...] - ETA: 15s - loss: 4.2837 - acc: 0.0307

KeyboardInterrupt: 

**Evaluate on testing set**

In [ ]:
ct = np.zeros((20,len(x_test),100))
for i in range(len(x_test)):
    for j in range(20):
        ct[j,i] = C2K_dot[j]
        
model.evaluate([x_test, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
          y_test_adj, batch_size=64, verbose=1)